In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import  sys


In [2]:
driver = webdriver.Chrome('C:/Users/HP/Downloads/chromedriver.exe')

In [3]:
# Function that parser through page and returns bs content
def parse_page(url):
    try:
        driver.get(url)
        content = driver.page_source
        soup = bs(content,'html.parser')
        return soup
    except:
        pass

In [4]:
# Returns a list of all models of one brand
def get_models(brand):
	brand = (brand.replace(' ', '-')).lower()
	url ='https://www.polovniautomobili.com/auto-oglasi/pretraga?brand=' + brand
	soup = parse_page(url)
	models= soup.find(id='model')
	models_list=[]
	for model in models:
		if model.get_text()=='Ostalo':
			pass
		else:
			models_list.append(model.get_text())
	return models_list[1:]

In [5]:
def replace(string):
	string = string.replace('š','s')
	string = string.replace('Š','s')
	string = string.replace('ć','c')
	string = string.replace('Ć','c')
	string = string.replace('Ž','z')
	string = string.replace('ž','z')
	string = string.replace('č','c')
	string = string.replace('Č','c')
	return string


In [6]:
def get_num_of_cars(url):
	soup = parse_page(url)
	if soup!=None:
		try:
			num = soup.findAll('small')
			num = num[3].get_text()
			num = num.split(' ')[-1]
			return (round(int(num)/25)+1)
		except:
			return -1
			pass

In [7]:
# Get all cars from one brand (iterate through each model in given brand)
def get_cars(brand, model):

    brand = (brand.replace(' ', '-')).lower()
    model = (model.replace(' ', '-')).lower()
    url ='https://www.polovniautomobili.com/auto-oglasi/pretraga?brand=' + brand + '&model[]=' + model
    list_of_links = []

    # find number of pages in a given model
    num = get_num_of_cars(url)
    if num==0:
        num=1
    # if ther are car ads on site and number of cars is greater or equal to 100
    if num!=-1 and num>=4:
        carList = []
        # loop over number of pages
        for i in range(1,num+1):
            soup = parse_page(url+'&page='+str(i))
            if soup!=None:
                articles = driver.find_elements_by_css_selector('a.ga-title')
                try:
                    for article in articles:
                        
                        list_of_links.append(article.get_attribute("href"))
                    return list_of_links
                except:
                    pass
            
                   
        #return list_of_links         
    else:
        pass


In [8]:
# function that inserts cars into dataframe and then loads into .csv file
def insertAll():
    # iterate through every brand and every model of that brand and load data into dataframe
    #brands = all_Brands()
    brands=['AUDI']

    try:
        for brand in brands:
            models = ['A3','A4']
            for model in models:
                if model==None:
                    pass
                else:
                    cars = get_cars(brand, model)
                    return cars
                    
                               
        #return cars
    except Exception as e:
        pass
    

In [9]:
carList=[]

def scrape_pages(urls):
    data = pd.DataFrame()

    
    for url in urls:
        try:
            driver.get(url)
            content = driver.page_source
            soup = bs(content,'html.parser')
            try:
                cena = soup.find(class_="price-item position-relative")
                cena_popust = soup.find("price-item-discount position-relative")
                if cena.get_text()!=None:
                    cena=cena.get_text()
                else:
                    cena=cena_popust.get_text()
                
                opste= soup.find_all(class_="uk-width-large-1-1 uk-width-medium-3-10 uk-width-1-2")
                karakteristike_naziv = soup.find_all(class_="uk-width-medium-1-4 uk-width-1-2")
                karakteristike = soup.find_all(class_="uk-width-medium-1-4 uk-width-1-2 uk-text-bold")
                oprema = soup.find_all(class_="uk-width-medium-1-3 uk-width-1-2")
    
                try:
                    karakteristike_list=[op.get_text() for op in opste] 
                    for k in karakteristike:
                        karakteristike_list.append(k.get_text())
                    oprema_num=len([opr.get_text() for opr in oprema])
                except: 
                    pass
                karakteristike_list.append(oprema_num)
                
                cena=cena.replace("\\n","")
                
                cena = cena[:-4].replace(".","")
                cena = int(cena[1:])
                
                karakteristike_list.append(cena)

                karakteristike_naziv_list = [kar.get_text() for kar in karakteristike_naziv]
                
                
                
                karakteristike_list.append(oprema_num)
                
                
                labels = ['Vozilo','Model','Marka','Godiste','Kilometraza','Karoserija', 'Gorivo', 'Kubikaza','Snaga', 'Fiksna cena','Zamena', 'Broj oglasa']
                for i in karakteristike_naziv_list:
                    labels.append(i)
                labels.append('Oprema')
                labels.append('Cena')
                dictionary=dict(zip(labels, karakteristike_list))
                carList.append(dictionary)
        
                
            except:
                print("Unexpected error:", sys.exc_info())
                pass
        except:
            print("Unexpected error:", sys.exc_info())
            pass
    data = data.append(carList,ignore_index=True)
    return data
                    
        

In [10]:

list_of_links = insertAll()
data = scrape_pages(list_of_links)


TypeError: 'NoneType' object is not iterable

In [ ]:
data.head()